## Exercise 1

Consider the following data:

| Tid | Refund | Marital Status | Taxable Income (K) | Cheat |
|-----|--------|----------------|--------------------|-------|
| 1   | Yes    | Single         | 125                | No    |
| 2   | No     | Married        | 100                | No    |
| 3   | No     | Single         | 70                 | No    |
| 4   | Yes    | Married        | 120                | No    |
| 5   | No     | Divorced       | 95                 | Yes   |
| 6   | No     | Married        | 60                 | No    |
| 7   | Yes    | Divorced       | 220                | No    |
| 8   | No     | Single         | 85                 | Yes   |
| 9   | No     | Married        | 75                 | No    |
| 10  | No     | Single         | 90                 | Yes   |

Complete the code below to find the best split. 

In [3]:

import pandas as pd
import numpy as np

# Sample data provided
data = {
    'Tid': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Refund': ['Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No'],
    'Marital Status': ['Single', 'Married', 'Single', 'Married', 'Divorced', 'Married', 'Divorced', 'Single', 'Married', 'Single'],
    'Taxable Income (K)': [125, 100, 70, 120, 95, 60, 220, 85, 75, 90],
    'Cheat': ['No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes']
}

# Create a DataFrame
df = pd.DataFrame(data)

# @TODO Function to calculate Gini impurity
def gini_impurity(groups, classes):

    """
    Calculate the Gini impurity for a split.
    
    Parameters:
    - groups: list of arrays/series, where each element contains the class labels for a group
    - classes: list of unique class labels
    
    Returns:
    - weighted Gini impurity for the split
    """
    # Total number of instances
    total_instances = sum(len(group) for group in groups)
    
    # Calculate weighted Gini impurity
    gini = 0.0
    
    for group in groups:
        size = len(group)
        
        # Avoid division by zero
        if size == 0:
            continue
        
        # Calculate Gini impurity for this group
        score = 0.0
        for class_val in classes:
            # Proportion of class_val in this group
            proportion = list(group).count(class_val) / size
            score += proportion ** 2
        
        # Gini impurity = 1 - sum(p^2)
        group_gini = 1.0 - score
        
        # Weight the group's Gini by its relative size
        gini += group_gini * (size / total_instances)
    
    return gini
# Split by 'Refund' attribute
refund_yes = df[df['Refund'] == 'Yes']['Cheat']
refund_no = df[df['Refund'] == 'No']['Cheat']
groups = [refund_yes, refund_no]
classes = ['Yes', 'No']

gini_score = gini_impurity(groups, classes)
print(f"Gini Impurity for 'Refund' split: {gini_score:.4f}")



# @TODO Function to split a dataset based on a column and a value
# Should return a tuple of left and right arrays
# For numeric columns, left should be <= the value
# For categorical columns, left should be == the value
def test_split(column, value, df):
    left, right = [], []
    # Check if the column is numeric or categorical
    if pd.api.types.is_numeric_dtype(df[column]):
        # Numeric split: left <= value, right > value
        left = df[df[column] <= value]
        right = df[df[column] > value]
    else:
        # Categorical split: left == value, right != value
        left = df[df[column] == value]
        right = df[df[column] != value]
    
    return left, right

# Function to get quartiles for continuous variables
def get_quartiles(series):
    return np.percentile(series, [25, 50, 75])

# Function to find the best split
def get_best_split(df_subset):
    dataset = df_subset.values.tolist()  # Convert dataframe subset to list of lists
    class_values = list(df_subset['Cheat'].unique())  # Unique class labels in this subset
    best_column, best_value, best_gini, best_groups = None, None, float('inf'), None
    
    # Iterate over each column (feature)
    for column in df_subset.columns[:-1]:  # Exclude the target variable 'Cheat'
        if df_subset[column].dtype == 'object':  # Categorical column
            unique_values = df_subset[column].unique()
            for value in unique_values:
                groups = test_split(df_subset.columns.get_loc(column), value, dataset)
                if len(groups[0]) == 0 or len(groups[1]) == 0:
                    continue
                gini = gini_impurity(groups, class_values)
                if gini < best_gini:
                    best_column, best_value, best_gini, best_groups = column, value, gini, groups
        
        elif np.issubdtype(df_subset[column].dtype, np.number):  # Numeric column
            # Use quartiles of the current subset
            quartiles = get_quartiles(df_subset[column])
            for value in quartiles:  # Try splitting by quartiles
                groups = test_split(df_subset.columns.get_loc(column), value, dataset)
                if len(groups[0]) == 0 or len(groups[1]) == 0:
                    continue
                gini = gini_impurity(groups, class_values)
                if gini < best_gini:
                    best_column, best_value, best_gini, best_groups = column, value, gini, groups
    
    return {'column': best_column, 'value': best_value, 'gini': best_gini, 'groups': best_groups}

# Function to split a DataFrame based on a column and value
def split_dataframe(df_subset, column, value):
    """Split a DataFrame into two based on a column and value"""
    if isinstance(value, (int, float)):
        left = df_subset[df_subset[column] <= value]
        right = df_subset[df_subset[column] > value]
    else:
        left = df_subset[df_subset[column] == value]
        right = df_subset[df_subset[column] != value]
    return left, right

# Function to find the best split
def get_best_split(df, target_column='Cheat'):
    """
    Find the best split for a DataFrame by trying all possible splits.
    
    Parameters:
    - df: pandas DataFrame
    - target_column: the column containing class labels (default 'Cheat')
    
    Returns:
    - dictionary with best split information
    """
    best_gini = float('inf')
    best_split = {}
    # Get unique classes
    classes = df[target_column].unique()
    
    # Try splitting on each column (except Tid and the target column)
    for column in df.columns:
        if column in ['Tid', target_column]:
            continue
        
        # Get unique values in this column
        unique_values = df[column].unique()
        
        # For numeric columns, try splitting at each unique value
        if pd.api.types.is_numeric_dtype(df[column]):
            for value in unique_values:
                left, right = split_dataframe(df, column, value)
                
                # Skip if split results in empty groups
                if len(left) == 0 or len(right) == 0:
                    continue
                
                # Calculate Gini impurity
                groups = [left[target_column], right[target_column]]
                gini = gini_impurity(groups, classes)
                
                # Update best split if this is better
                if gini < best_gini:
                    best_gini = gini
                    best_split = {
                        'column': column,
                        'value': value,
                        'gini': gini,
                        'left_size': len(left),
                        'right_size': len(right)
                    }
        
        # For categorical columns, try each unique value
        else:
            for value in unique_values:
                left, right = split_dataframe(df, column, value)
                
                # Skip if split results in empty groups
                if len(left) == 0 or len(right) == 0:
                    continue
                
                # Calculate Gini impurity
                groups = [left[target_column], right[target_column]]
                gini = gini_impurity(groups, classes)
                
                # Update best split if this is better
                if gini < best_gini:
                    best_gini = gini
                    best_split = {
                        'column': column,
                        'value': value,
                        'gini': gini,
                        'left_size': len(left),
                        'right_size': len(right)
                    }
    
    return best_split

# Call the function to find the best split
best_split = get_best_split(df)
print('Best Split: Column', best_split['column'], 'with value', best_split['value'], 'gini score =', best_split['gini'])
print(f"Left group size: {best_split['left_size']}, Right group size: {best_split['right_size']}")




Gini Impurity for 'Refund' split: 0.3429
Best Split: Column Marital Status with value Married gini score = 0.3
Left group size: 4, Right group size: 6


Brief Explanation of Results:
1. Refund Split (Gini = 0.3429)

Splitting by "Refund" creates moderately mixed groups
Not the best feature for predicting who cheats

2. Best Split: Marital Status = "Married" (Gini = 0.3000)

This is the optimal first split for the decision tree
Marital Status is better than Refund for separating cheaters from non-cheaters
Lower Gini (0.30 vs 0.34) = better separation

3. Group Sizes:

Left group (Married): 4 people
Right group (Not Married): 6 people

### Exercise 3

Try using the preceding code to implement a complete decision tree algorithm! You will need to use a list to keep track of where you are in the tree.

**Hint 1: Think of it like a "to-do list"**

- Start with your full dataset as your first "task"
- When you split a dataset, you create two new "tasks" (left and right groups)
- Keep these tasks in a list - what happens when you add new tasks to the end and always work on the last task? Try it with a simple example on paper first!

**Hint 2: What information do you need to remember?**

For each "task" in your list, you'll need to track:

- The subset of data you're working with
- How deep you are in the tree (start at depth 0)
- The conditions that got you here (like "Refund == 'Yes' AND Income <= 85")

**Hint 3: When do you stop splitting?**

Think about when it doesn't make sense to split further:

- When all samples in a group have the same class (all "Cheat" or all "Don't Cheat")
- When you've gone too deep (set a maximum depth like 3)
- When you have too few samples to split meaningfully

**Hint 4: Process one group completely before moving to another**

- Use .pop() to take the last item from your to-do list
- This naturally makes you finish one "branch" before starting another
- When you can't split anymore, print out the rule you've discovered!

**Hint 5: Building the rule strings**

- As you go deeper, add conditions to a list: ["Refund == 'Yes'", "Income <= 85"]
- When you reach a leaf (can't split anymore), combine them: "IF Refund == 'Yes' AND Income <= 85 THEN Cheat = 'No'"

In [4]:
# Simple depth-first tree builder using DataFrame subsetting
def build_tree_dfs(df, max_depth=3, min_samples=2):
    """Build a decision tree using a simple stack-based DFS approach"""
    
    # Initialize stack with: (df_subset, depth, path_conditions)
    stack = []
    stack.append((df, 0, []))
    
    rule_number = 1
    
    while stack:
        df_subset, depth, conditions = stack.pop()
        
        # Get class distribution
        class_counts = df_subset['Cheat'].value_counts().to_dict()
        unique_classes = list(class_counts.keys())
        
        print(f"\n{'  ' * depth}Level {depth}: {len(df_subset)} samples")
        print(f"{'  ' * depth}Class distribution: {class_counts}")

    # THE REST IS UP TO YOU!
# Check stopping conditions
        # 1. Pure node (only one class)
        if len(unique_classes) == 1:
            print(f"{'  ' * depth}→ LEAF: Predict '{unique_classes[0]}'")
            print(f"{'  ' * depth}   Rule #{rule_number}: IF {' AND '.join(conditions) if conditions else 'root'} THEN Cheat = '{unique_classes[0]}'")
            rule_number += 1
            continue
        
        # 2. Max depth reached
        if depth >= max_depth:
            majority_class = max(class_counts, key=class_counts.get)
            print(f"{'  ' * depth}→ LEAF (max depth): Predict '{majority_class}'")
            print(f"{'  ' * depth}   Rule #{rule_number}: IF {' AND '.join(conditions) if conditions else 'root'} THEN Cheat = '{majority_class}'")
            rule_number += 1
            continue
        
        # 3. Too few samples
        if len(df_subset) < min_samples:
            majority_class = max(class_counts, key=class_counts.get)
            print(f"{'  ' * depth}→ LEAF (min samples): Predict '{majority_class}'")
            print(f"{'  ' * depth}   Rule #{rule_number}: IF {' AND '.join(conditions) if conditions else 'root'} THEN Cheat = '{majority_class}'")
            rule_number += 1
            continue
        
        # Find the best split for this subset
        best_split = get_best_split(df_subset)
        
        # 4. No valid split found
        if not best_split:
            majority_class = max(class_counts, key=class_counts.get)
            print(f"{'  ' * depth}→ LEAF (no split): Predict '{majority_class}'")
            print(f"{'  ' * depth}   Rule #{rule_number}: IF {' AND '.join(conditions) if conditions else 'root'} THEN Cheat = '{majority_class}'")
            rule_number += 1
            continue
        
        # Display the split
        print(f"{'  ' * depth}Split on '{best_split['column']}' with value '{best_split['value']}' (Gini = {best_split['gini']:.4f})")
        
        # Split the data
        left, right = split_dataframe(df_subset, best_split['column'], best_split['value'])
        
        # Create condition strings
        if isinstance(best_split['value'], (int, float)):
            left_condition = f"{best_split['column']} <= {best_split['value']}"
            right_condition = f"{best_split['column']} > {best_split['value']}"
        else:
            left_condition = f"{best_split['column']} == '{best_split['value']}'"
            right_condition = f"{best_split['column']} != '{best_split['value']}'"
        
        # Push right child first (so left is processed first - DFS left-to-right)
        if len(right) > 0:
            right_conditions = conditions + [right_condition]
            stack.append((right, depth + 1, right_conditions))
        
        if len(left) > 0:
            left_conditions = conditions + [left_condition]
            stack.append((left, depth + 1, left_conditions))

build_tree_dfs(df)


Level 0: 10 samples
Class distribution: {'No': 7, 'Yes': 3}
Split on 'Marital Status' with value 'Married' (Gini = 0.3000)

  Level 1: 4 samples
  Class distribution: {'No': 4}
  → LEAF: Predict 'No'
     Rule #1: IF Marital Status == 'Married' THEN Cheat = 'No'

  Level 1: 6 samples
  Class distribution: {'No': 3, 'Yes': 3}
  Split on 'Refund' with value 'Yes' (Gini = 0.2500)

    Level 2: 2 samples
    Class distribution: {'No': 2}
    → LEAF: Predict 'No'
       Rule #2: IF Marital Status != 'Married' AND Refund == 'Yes' THEN Cheat = 'No'

    Level 2: 4 samples
    Class distribution: {'Yes': 3, 'No': 1}
    Split on 'Taxable Income (K)' with value '70' (Gini = 0.0000)

      Level 3: 1 samples
      Class distribution: {'No': 1}
      → LEAF: Predict 'No'
         Rule #3: IF Marital Status != 'Married' AND Refund != 'Yes' AND Taxable Income (K) == '70' THEN Cheat = 'No'

      Level 3: 3 samples
      Class distribution: {'Yes': 3}
      → LEAF: Predict 'Yes'
         Rule #4: I

## Exercise 2

Build a decision tree to fit the [federalist papers](https://www.kaggle.com/datasets/tobyanderson/federalist-papers_) data, available in the data directory (click on the link to find out more information about this data). Note that you should restrict your analysis to papers by Hamilton or Madison.  Plot your training and test scores to pick a value for ccp_alpha. What did you pick?  Run your trained classifier on the "disputed" papers.  What does your model tell you? 